In [25]:
import numpy as np
import pandas as pd
import astropy.io.fits as fits
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
import matplotlib.pyplot as plt
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
import umap
import plotly.express as px

# コスモロジーモデルの設定
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# FITSファイルのパス
fits_file_path = '../COSMOS2020_CLASSIC_R1_v2.2_p3.fits'

# フィルタリング用のカラムリスト
column = ['ez_z_phot', 'ez_z_phot_risk', 'GALEX_FUV_FLUX', 'GALEX_NUV_FLUX', 'CFHT_u_FLUX_AUTO', 'CFHT_ustar_FLUX_AUTO', 'ACS_F814W_FLUX', 'HSC_g_FLUX_AUTO', 'HSC_r_FLUX_AUTO', 'HSC_i_FLUX_AUTO', 'HSC_z_FLUX_AUTO', 'HSC_y_FLUX_AUTO', 'SC_B_FLUX_AUTO', 'SC_gp_FLUX_AUTO', 
    'SC_V_FLUX_AUTO', 'SC_rp_FLUX_AUTO', 
    'SC_ip_FLUX_AUTO', 'SC_zp_FLUX_AUTO', 
    'SC_zpp_FLUX_AUTO', 'SC_IB427_FLUX_AUTO', 
    'SC_IB464_FLUX_AUTO', 'SC_IA484_FLUX_AUTO', 
    'SC_IB505_FLUX_AUTO', 'SC_IA527_FLUX_AUTO', 
    'SC_IB574_FLUX_AUTO', 'SC_IA624_FLUX_AUTO', 
    'SC_IA679_FLUX_AUTO', 'SC_IB709_FLUX_AUTO', 
    'SC_IA738_FLUX_AUTO', 'SC_IA767_FLUX_AUTO', 
    'SC_IB827_FLUX_AUTO', 'SC_NB711_FLUX_AUTO', 
    'SC_NB816_FLUX_AUTO', 'UVISTA_Y_FLUX_AUTO', 'UVISTA_J_FLUX_AUTO', 'UVISTA_H_FLUX_AUTO', 'UVISTA_Ks_FLUX_AUTO', 'UVISTA_NB118_FLUX_AUTO', 'SPLASH_CH1_FLUX', 'SPLASH_CH2_FLUX', 'SPLASH_CH3_FLUX', 'SPLASH_CH4_FLUX']

# FITSファイルを開く
with fits.open(fits_file_path) as hdul:
    data = hdul[1].data
    
    # データをPandas DataFrameに変換
    df = pd.DataFrame(data)
    
    # フィルタリングと前処理
    df = df[df['ez_z_phot'] <= 2]
    df = df.dropna(subset=column)
    df = df[df['ez_z_phot_risk'] < 0.5]
    df = df.sort_values(by='ez_z_phot')

# 光度距離を計算して追加
df['d_L'] = df['ez_z_phot'].apply(lambda z: cosmo.luminosity_distance(z).to(u.cm).value)

# 新しい列を一度に計算して追加
new_columns = {}
for col in column:
    if '_FLUX' in col:
        new_columns[col + '_LUMINOSITY'] = 4 * np.pi * (df['d_L']**2) * df[col] * 1e-29  # μJyからerg/s/Hzに変換

# 新しい列を一度に追加
df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)


# NumPyファイルに保存
# numpy_file_path = 'filtered_data.npy'
# np.save(numpy_file_path, df.to_numpy())

# print(f'フィルタリングされたデータをNumPyファイルに保存しました: {numpy_file_path}')

In [26]:
# # カラム名をテキストファイルに書き込む
# txt_file_path = 'columns.txt'
# with open(txt_file_path, 'w') as file:
#     for column in df.columns:
#         file.write(column + '\n')

# print(f'カラム名をテキストファイルに保存しました: {txt_file_path}')

In [27]:
print(df.columns)

Index(['ID', 'ALPHA_J2000', 'DELTA_J2000', 'X_IMAGE', 'Y_IMAGE', 'ERRX2_IMAGE',
       'ERRY2_IMAGE', 'ERRXY_IMAGE', 'FLUX_RADIUS', 'KRON_RADIUS',
       ...
       'SC_NB816_FLUX_AUTO_LUMINOSITY', 'UVISTA_Y_FLUX_AUTO_LUMINOSITY',
       'UVISTA_J_FLUX_AUTO_LUMINOSITY', 'UVISTA_H_FLUX_AUTO_LUMINOSITY',
       'UVISTA_Ks_FLUX_AUTO_LUMINOSITY', 'UVISTA_NB118_FLUX_AUTO_LUMINOSITY',
       'SPLASH_CH1_FLUX_LUMINOSITY', 'SPLASH_CH2_FLUX_LUMINOSITY',
       'SPLASH_CH3_FLUX_LUMINOSITY', 'SPLASH_CH4_FLUX_LUMINOSITY'],
      dtype='object', length=795)


In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
import plotly.express as px

# データフレームの読み込みや定義
# df = pd.read_csv('your_data.csv')
# column = ['list_of_flux_columns']

# HTMLファイルを作成
html_file = 'KernelPCA_results_with_toggle_SFR.html'

# HTMLファイルの開始部分を定義
html_start = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Plot Toggle</title>
    <script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>
    <style>
        .plot-container {
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>Interactive Plot Toggle</h1>
"""

# プロットを格納するリスト
figures = []
toggle_div_template = """
    <div class="plot-container">
        <button onclick="togglePlot('{plot_id}')">Toggle Plot z={plot_number_down}-{plot_number_up}</button>
        <div id="{plot_id}" style="display: none;">
            {plot_content}
        </div>
    </div>
"""

# 赤方偏移ごとに処理
for i, z in enumerate(np.arange(0, 2.1, 0.1)):
    z_min = z
    z_max = z + 0.1
    df_subset = df[(df['ez_z_phot'] >= z_min) & (df['ez_z_phot'] < z_max)]
    
    if df_subset.empty:
        continue
    
    if len(df_subset) > 20:
        color_max = df_subset['ez_sfr'].sort_values(ascending=False).iloc[20]
    else:
        color_max = df_subset['ez_sfr'].max()
    
    color_min = df_subset['ez_sfr'].min()

    # データのスケーリング
    X = df_subset[[col + '_LUMINOSITY' for col in column if '_FLUX' in col]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # KernelPCAの適用
    kpca = KernelPCA(n_components=3, kernel='rbf')
    X_kpca = kpca.fit_transform(X_scaled)
    
    df_kpca = pd.DataFrame(X_kpca, columns=['KPCA1', 'KPCA2', 'KPCA3'])
    df_kpca['SFR'] = df_subset['ez_sfr'].values
    
    fig = px.scatter_3d(df_kpca, x='KPCA1', y='KPCA2', z='KPCA3', color='SFR',
                        color_continuous_scale='Viridis',
                        range_color=[color_min, color_max],
                        title=f'KernelPCA for z={z_min:.1f} to z={z_max:.1f}')
    fig.update_traces(marker=dict(size=2))
    
    # プロットをリストに追加
    fig_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    figures.append(toggle_div_template.format(plot_id=f'plot_{i}', plot_number_down=f'{(i+1)*0.1-0.1:.1f}', plot_number_up=f'{(i+1)*0.1:.1f}', plot_content=fig_html))

# HTMLファイルの終了部分を定義
html_end = """
    <script>
        function togglePlot(plotId) {
            var plotDiv = document.getElementById(plotId);
            if (plotDiv.style.display === "none") {
                plotDiv.style.display = "block";
            } else {
                plotDiv.style.display = "none";
            }
        }
    </script>
</body>
</html>
"""

# HTMLファイルにすべてのプロットを保存
with open(html_file, 'w', encoding='utf-8') as f:
    f.write(html_start)
    for fig_div in figures:
        f.write(fig_div)
    f.write(html_end)

print(f'All plots have been saved to {html_file}')


All plots have been saved to KernelPCA_results_with_toggle_SFR.html


In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
import plotly.express as px

# データフレームの読み込みや定義
# df = pd.read_csv('your_data.csv')
# column = ['list_of_flux_columns']

# HTMLファイルを作成
html_file = 'KernelPCA_results_with_toggle_MASS.html'

# HTMLファイルの開始部分を定義
html_start = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Plot Toggle</title>
    <script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>
    <style>
        .plot-container {
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>Interactive Plot Toggle</h1>
"""

# プロットを格納するリスト
figures = []
toggle_div_template = """
    <div class="plot-container">
        <button onclick="togglePlot('{plot_id}')">Toggle Plot z={plot_number_down}-{plot_number_up}</button>
        <div id="{plot_id}" style="display: none;">
            {plot_content}
        </div>
    </div>
"""

# 赤方偏移ごとに処理
for i, z in enumerate(np.arange(0, 2.1, 0.1)):
    z_min = z
    z_max = z + 0.1
    df_subset = df[(df['ez_z_phot'] >= z_min) & (df['ez_z_phot'] < z_max)]
    
    if df_subset.empty:
        continue
    
    if len(df_subset) > 20:
        color_max = df_subset['ez_mass'].sort_values(ascending=False).iloc[20]
    else:
        color_max = df_subset['ez_mass'].max()
    
    color_min = df_subset['ez_mass'].min()

    # データのスケーリング
    X = df_subset[[col + '_LUMINOSITY' for col in column if '_FLUX' in col]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # KernelPCAの適用
    kpca = KernelPCA(n_components=3, kernel='rbf')
    X_kpca = kpca.fit_transform(X_scaled)
    
    df_kpca = pd.DataFrame(X_kpca, columns=['KPCA1', 'KPCA2', 'KPCA3'])
    df_kpca['MASS'] = df_subset['ez_mass'].values
    
    fig = px.scatter_3d(df_kpca, x='KPCA1', y='KPCA2', z='KPCA3', color='MASS',
                        color_continuous_scale='Viridis',
                        range_color=[color_min, color_max],
                        title=f'KernelPCA for z={z_min:.1f} to z={z_max:.1f}')
    fig.update_traces(marker=dict(size=2))
    
    # プロットをリストに追加
    fig_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    figures.append(toggle_div_template.format(plot_id=f'plot_{i}', plot_number_down=f'{(i+1)*0.1-0.1:.1f}', plot_number_up=f'{(i+1)*0.1:.1f}', plot_content=fig_html))

# HTMLファイルの終了部分を定義
html_end = """
    <script>
        function togglePlot(plotId) {
            var plotDiv = document.getElementById(plotId);
            if (plotDiv.style.display === "none") {
                plotDiv.style.display = "block";
            } else {
                plotDiv.style.display = "none";
            }
        }
    </script>
</body>
</html>
"""

# HTMLファイルにすべてのプロットを保存
with open(html_file, 'w', encoding='utf-8') as f:
    f.write(html_start)
    for fig_div in figures:
        f.write(fig_div)
    f.write(html_end)

print(f'All plots have been saved to {html_file}')


All plots have been saved to KernelPCA_results_with_toggle_MASS.html


In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
import plotly.express as px

# データフレームの読み込みや定義
# df = pd.read_csv('your_data.csv')
# column = ['list_of_flux_columns']

# HTMLファイルを作成
html_file = 'KernelPCA_results_with_toggle_Av.html'

# HTMLファイルの開始部分を定義
html_start = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Plot Toggle</title>
    <script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>
    <style>
        .plot-container {
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>Interactive Plot Toggle</h1>
"""

# プロットを格納するリスト
figures = []
toggle_div_template = """
    <div class="plot-container">
        <button onclick="togglePlot('{plot_id}')">Toggle Plot z={plot_number_down}-{plot_number_up}</button>
        <div id="{plot_id}" style="display: none;">
            {plot_content}
        </div>
    </div>
"""

# 赤方偏移ごとに処理
for i, z in enumerate(np.arange(0, 2.1, 0.1)):
    z_min = z
    z_max = z + 0.1
    df_subset = df[(df['ez_z_phot'] >= z_min) & (df['ez_z_phot'] < z_max)]
    
    if df_subset.empty:
        continue
    
    if len(df_subset) > 20:
        color_max = df_subset['ez_Av'].sort_values(ascending=False).iloc[20]
    else:
        color_max = df_subset['ez_Av'].max()
    
    color_min = df_subset['ez_Av'].min()

    # データのスケーリング
    X = df_subset[[col + '_LUMINOSITY' for col in column if '_FLUX' in col]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # KernelPCAの適用
    kpca = KernelPCA(n_components=3, kernel='rbf')
    X_kpca = kpca.fit_transform(X_scaled)
    
    df_kpca = pd.DataFrame(X_kpca, columns=['KPCA1', 'KPCA2', 'KPCA3'])
    df_kpca['Av'] = df_subset['ez_Av'].values
    
    fig = px.scatter_3d(df_kpca, x='KPCA1', y='KPCA2', z='KPCA3', color='Av',
                        color_continuous_scale='Viridis',
                        range_color=[color_min, color_max],
                        title=f'KernelPCA for z={z_min:.1f} to z={z_max:.1f}')
    fig.update_traces(marker=dict(size=2))
    
    # プロットをリストに追加
    fig_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    figures.append(toggle_div_template.format(plot_id=f'plot_{i}', plot_number_down=f'{(i+1)*0.1-0.1:.1f}', plot_number_up=f'{(i+1)*0.1:.1f}', plot_content=fig_html))

# HTMLファイルの終了部分を定義
html_end = """
    <script>
        function togglePlot(plotId) {
            var plotDiv = document.getElementById(plotId);
            if (plotDiv.style.display === "none") {
                plotDiv.style.display = "block";
            } else {
                plotDiv.style.display = "none";
            }
        }
    </script>
</body>
</html>
"""

# HTMLファイルにすべてのプロットを保存
with open(html_file, 'w', encoding='utf-8') as f:
    f.write(html_start)
    for fig_div in figures:
        f.write(fig_div)
    f.write(html_end)

print(f'All plots have been saved to {html_file}')


All plots have been saved to KernelPCA_results_with_toggle_Av.html


In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
import plotly.express as px

# データフレームの読み込みや定義
# df = pd.read_csv('your_data.csv')
# column = ['list_of_flux_columns']

# HTMLファイルを作成
html_file = 'KernelPCA_results_with_toggle_LHa.html'

# HTMLファイルの開始部分を定義
html_start = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Plot Toggle</title>
    <script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>
    <style>
        .plot-container {
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>Interactive Plot Toggle</h1>
"""

# プロットを格納するリスト
figures = []
toggle_div_template = """
    <div class="plot-container">
        <button onclick="togglePlot('{plot_id}')">Toggle Plot z={plot_number_down}-{plot_number_up}</button>
        <div id="{plot_id}" style="display: none;">
            {plot_content}
        </div>
    </div>
"""

# 赤方偏移ごとに処理
for i, z in enumerate(np.arange(0, 2.1, 0.1)):
    z_min = z
    z_max = z + 0.1
    df_subset = df[(df['ez_z_phot'] >= z_min) & (df['ez_z_phot'] < z_max)]
    
    if df_subset.empty:
        continue
    
    if len(df_subset) > 20:
        color_max = df_subset['ez_LHa'].sort_values(ascending=False).iloc[20]
    else:
        color_max = df_subset['ez_LHa'].max()
    
    color_min = df_subset['ez_LHa'].min()

    # データのスケーリング
    X = df_subset[[col + '_LUMINOSITY' for col in column if '_FLUX' in col]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # KernelPCAの適用
    kpca = KernelPCA(n_components=3, kernel='rbf')
    X_kpca = kpca.fit_transform(X_scaled)
    
    df_kpca = pd.DataFrame(X_kpca, columns=['KPCA1', 'KPCA2', 'KPCA3'])
    df_kpca['LHa'] = df_subset['ez_LHa'].values
    
    fig = px.scatter_3d(df_kpca, x='KPCA1', y='KPCA2', z='KPCA3', color='LHa',
                        color_continuous_scale='Viridis',
                        range_color=[color_min, color_max],
                        title=f'KernelPCA for z={z_min:.1f} to z={z_max:.1f}')
    fig.update_traces(marker=dict(size=2))
    
    # プロットをリストに追加
    fig_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    figures.append(toggle_div_template.format(plot_id=f'plot_{i}', plot_number_down=f'{(i+1)*0.1-0.1:.1f}', plot_number_up=f'{(i+1)*0.1:.1f}', plot_content=fig_html))

# HTMLファイルの終了部分を定義
html_end = """
    <script>
        function togglePlot(plotId) {
            var plotDiv = document.getElementById(plotId);
            if (plotDiv.style.display === "none") {
                plotDiv.style.display = "block";
            } else {
                plotDiv.style.display = "none";
            }
        }
    </script>
</body>
</html>
"""

# HTMLファイルにすべてのプロットを保存
with open(html_file, 'w', encoding='utf-8') as f:
    f.write(html_start)
    for fig_div in figures:
        f.write(fig_div)
    f.write(html_end)

print(f'All plots have been saved to {html_file}')


All plots have been saved to KernelPCA_results_with_toggle_LHa.html
